In [1]:
import pandas as pd
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.experimental import QTradableStocksUS
import numpy as np

# Función de utilidad para obtener el universo de empresas en quantopian

In [2]:
def make_pipeline():
    universe = QTradableStocksUS()
    market_cap = Fundamentals.market_cap.latest
    return Pipeline(
        columns={
            'market_cap':market_cap,
        }, screen=universe
    )

# Extracción del universo de empresas en los periodos correspondientes

In [3]:
past_date = '2019-10-04'
past =run_pipeline(make_pipeline(),start_date=past_date,end_date=past_date)
present_date = '2020-04-03'
today = run_pipeline(make_pipeline(),start_date=present_date,end_date=present_date)

## Limpieza del output para mejor visualización

In [4]:
today.index = map(lambda x :str(x).split('[')[-1][:-3] , today.index.values )
past.index = map(lambda x :str(x).split('[')[-1][:-3] , past.index.values )

# Obtención de resultado en caso de una inversión hace un tiempo

In [5]:
# tomamos solo las empresas que estaban presentes en el pasado y que siguen presentes
empresas = list(set(list(today.index.values)).intersection(list(past.index.values)))

In [6]:
#juntamos ambos dataframes
df = pd.concat([today.loc[empresas],past.loc[empresas]], axis = 1)

In [7]:
# usamos el API de quantopian para obtener los datos de los stocks 
def get_stock_value(corp, date , low=True):
    df = get_pricing(symbols=corp,start_date=date ,end_date=date)
    return df.iloc[0].low if low else df.iloc[0].high
df['old_stock'] = [get_stock_value(x, past_date) for x in df.index]
df['current_stock'] = [get_stock_value(x, present_date) for x in df.index]

In [8]:
df['ganancia'] = df['current_stock'] / df['old_stock']
df ['sid'] = [symbols(c).sid for c in df.index]
df.sort_values('ganancia', ascending = False).head(7)

,market_cap,market_cap,old_stock,current_stock,ganancia,sid
AXSM,2.063618e+09,6.305479e+08,16.860,54.370,3.224792,49607
AUPH,1.616111e+09,4.840818e+08,5.080,14.250,2.805118,47621
TDOC,1.163168e+10,4.632927e+09,64.730,155.010,2.394717,49222
MRNA,1.092291e+10,4.975846e+09,14.890,32.720,2.197448,52751
EHTH,3.420454e+09,1.438605e+09,59.412,125.370,2.110180,32726
XLRN,4.458998e+09,2.086927e+09,38.820,80.098,2.063318,45431
TSLA,8.379931e+10,4.174202e+10,228.070,468.390,2.053712,39840
